In [147]:
import numpy as np
import scipy.spatial as spatial
import pandas as pd
import pickle
import os
from smart_open import smart_open
import gensim

In [148]:
%matplotlib inline

## To rebuild ground truth corpus from scratch (requires use of d2v scripts on repo)

In [18]:
groundTruthFile='/Users/eslt0101/Projects/EviDENce/GroundTruth/Manual_annotation_TOTAAL_door_Jeroen_en_Susan.xlsx'
groundTruthFileSheet='Resultaat Manuele Annotatie'
groundTruth = pd.read_excel(groundTruthFile, sheet_name=groundTruthFileSheet)


#groundTruth['Titel']


#type(groundTruth)

groundTruth.drop_duplicates(['Titel'],keep='first', inplace=True)
groundTruth.reset_index(drop=True,inplace=True)
#groundTruth['Titel'][121]


In [19]:
stopwordsFile = '/Users/eslt0101/Projects/EviDENce/ML/stopwords-nl-stopwords-iso_expanded_gensim_processed.txt'
corpus_dir = '/Users/eslt0101/Data/eScience/EviDENce/Data/NR-Teksts/EviDENce_NR_output_clean/TargetSize150'
corpus_token_file = 'GV_corpus_text_para_clean.txt'
corpus_text_file = 'GV_corpus_text_para_clean.txt'
corpus_filenames_file = 'GV_corpus_text_para_clean_filenames.txt'

full_ctokf = os.path.join(corpus_dir,corpus_token_file)
full_ctf = os.path.join(corpus_dir,corpus_text_file)
full_cff = os.path.join(corpus_dir,corpus_filenames_file) 

with open(stopwordsFile) as f:
    dutch_stopwords = f.readlines()
    

dutch_stopwords = [w.rstrip() for  w in dutch_stopwords]


def read_corpus(corpus_token_file,labeled=False):
    with smart_open(corpus_token_file,'r') as tf:
        for i,text_line in enumerate(tf):
            if labeled :
                #yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text_line), [fn_line.rstrip()])
                yield gensim.models.doc2vec.TaggedDocument([token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords], [i])
            else:
                yield [token for token in gensim.utils.simple_preprocess(text_line,deacc=True) if token not in dutch_stopwords]

def read_corpus_lookup(corpus_filenames_file, corpus_token_file) :
    with smart_open(corpus_filenames_file, 'r') as fnf, smart_open(corpus_token_file,'r') as tf:
        i=0
        for (fn_line,tf_line) in zip(fnf,tf):
            yield ([i],[fn_line.rstrip()],[tf_line])
            i+=1


corp = list(read_corpus(full_ctokf,labeled=True))


corp_lookup = list(read_corpus_lookup(full_cff,full_ctokf))

reinf_corp_medvec= np.load('/Users/eslt0101/Projects/EviDENce/ML/reinferred_corpus_terms_para_150_medvec_clean.npy')




In [20]:
with open(full_cff,'r') as fnf:
    corpFileNames=fnf.readlines()

corpFileNames = [fn.rstrip().split('.')[0] for fn in corpFileNames ]
#corpFileNames = [fn.rstrip() for fn in corpFileNames ]


groundTruth_inCorpusIds =[]
for j in range(len(groundTruth)):
    #print(j, groundTruth['Titel'][j])
    gtTitel = groundTruth['Titel'][j]
    corpusIndex = [i for i,ent in enumerate(corpFileNames) if ent == gtTitel]
    groundTruth_inCorpusIds.append(corpusIndex[0])



#groundTruth_inCorpusIds




gt_corp_lookup = [corp_lookup[i] for i in groundTruth_inCorpusIds]



gt_corp=[corp[i] for i in groundTruth_inCorpusIds]


In [21]:
gt_corp_reinf_medvec=reinf_corp_medvec[groundTruth_inCorpusIds]


### Append reinferred vectors to dataframe

series_reinfVec =pd.Series(np.array([]))
for i in range(len(gt_corp_reinf_medvec)):
    series_reinfVec[i] = gt_corp_reinf_medvec[i]
    


groundTruthEx = groundTruth



groundTruthEx.insert(10,'reinfVec',series_reinfVec)




## To import ground truth data set as pickle

In [149]:
groundTruthEx = pd.read_pickle('/Users/eslt0101/Projects/REPOS/eScience/EviDENce/doc2vec/d2v_evaluation/groundTruthEx_pd.pkl')#

In [150]:
groundTruthEx


,Herkomst,Titel,Tekst,JEROEN_janee,JEROEN_OPM,SUSAN_janee,SUSAN_OPM,VERGELIJKING,CONCLUSIE,EINDOORDEEL_janee,reinfVec,vecNorm
0,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Welling_c...,"Rotterdammer bent , maar dat ben ik dus niet ,...",nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.3392736315727234, 0.27179042994976044, -0.1...",1.346457
1,manual_annotation_sample_seed1111,GV_NMKV_Sachsenhausen_01_conversation_clipped_...,Op de blokken stond geschilderd : es gibt nur ...,ja,indirect,ja,twijfel,zelfde,NaN,ja,"[-0.16284064203500748, 0.10027723386883736, -0...",1.456656
2,manual_annotation_sample_seed1111,GV_NIOD_Buchenwald_36_clipped_150_paragraph_55...,En die een kader te ja . Dat was een aantal ap...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.042901355773210526, 0.13078030943870544, 0....",1.452613
3,manual_annotation_sample_seed1111,GV_Overloon_evacuees_08_conversation_clipped_1...,Die onderduiker die met die koe aankwam dat wa...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.2558271139860153, -0.16466810554265976, -0....",1.414626
4,manual_annotation_sample_seed1111,GV_Limburg_gastkind_03_conversation_clipped_15...,Hebben jullie er na de oorlog nog veel over ge...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[-0.032807089388370514, 0.14076124876737595, -...",1.523190
5,manual_annotation_sample_seed1111,GV_Zigma_koopvaardij_07_conversation_clipped_1...,In ieder konvooi had je een paar van die schee...,ja,NaN,ja,NaN,zelfde,NaN,ja,"[0.4917626976966858, 0.24750546365976334, -0.0...",1.980321
6,manual_annotation_sample_seed1111,GV_DeJager_ReisvandeRazzia_interview_Smits_con...,"En als je een schoen aan hebt , dat is een hee...",ja,NaN,ja,NaN,zelfde,NaN,ja,"[-0.006696479162201285, 0.1732935756444931, -0...",1.415925
7,manual_annotation_sample_seed1111,GV_SiteFilms_Java_05_conversation_clipped_150_...,Maar hoe ze heetten ... Dat is van korte duur ...,nee,NaN,nee,twijfel,zelfde,NaN,nee,"[0.045544352382421494, 0.2830629199743271, 0.1...",1.342135
8,manual_annotation_sample_seed1111,GV_KNMG_medici_11_conversation_clipped_150_par...,O u besprak de patiënten met Weterings ? Jazek...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.32386012375354767, 0.21032172441482544, -0....",1.762902
9,manual_annotation_sample_seed1111,GV_OVCG_Groningeninoorlogstijd_08_conversation...,Dus . Toen kwam er in een keer een hele andere...,nee,NaN,nee,NaN,zelfde,NaN,nee,"[0.06386139988899231, 0.307253435254097, -0.11...",1.165109


In [151]:
def retrieval_pass(inVec, corpDF, nReturn):
    """calculate cosine ditance of query vector inVec to all corpus vectors"""
    cosd_vec_corp = cosdistance_corp(inVec,corpDF['reinfVec'])
    
    """rank cosine distances in descending order"""
    ranked_retrieval = np.argsort(cosd_vec_corp)[::-1]
    
    "get cosine distances in ranked order"
    ranked_cosd = cosd_vec_corp[ranked_retrieval]
    ret_cosd = cosd_vec_corp[ranked_retrieval]
    
    
    
    """ return top nReturn instances"""
    ranking = corpDF.iloc[ranked_retrieval,[1,9,10]].copy()
    ranking['cosine_distance']= ret_cosd
    

    
    selected = ranking.iloc[0:nReturn].copy()
    """ create pandas series of selected ranked cosine distances"""
    #ret_cosd = pd.Series(ranked_cosd[0:nReturn])
    
    """insert cosine distance into returned ranking"""
    
    
    return selected
    
    
    

In [152]:
def cosdistance(v1,v2):
    d = np.array([1. - spatial.distance.cosine(v1,v2)])
    return d

In [153]:
def cosdistance_corp(vec,corpus_vectors):
    cosd_vec_corp =[]
    for i in range (len(corpus_vectors)):
        cd = cosdistance(vec,corpus_vectors[i])
        cosd_vec_corp.append(cd)
    cosd_vec_corp = np.array(cosd_vec_corp).flatten()
    return cosd_vec_corp

In [154]:
def calculate_PR_LR(retrievedDF):
    P_LR = len(np.array(np.where(retrievedDF['EINDOORDEEL_janee'] == 'ja')).flatten())/len(retrievedDF['EINDOORDEEL_janee'])
    R_LR = P_LR
    return P_LR, R_LR



In [155]:
def run_evaluation(corpDF,lenRetrieve):

    precisionLR =[]
    recallLR =[]
    queries =[]
    
    loc_yes = np.array(np.where(corpDF['EINDOORDEEL_janee'] == 'ja')).flatten()
    
    for i in range(len(loc_yes)):
        queryVec = corpDF['reinfVec'][loc_yes[i]]
        queryTitle = corpDF['Titel'][loc_yes[i]]
        
        incorp = corpDF.copy()
        incorp.drop([loc_yes[i]],inplace=True)
        incorp = incorp.reset_index(drop=True)
        
        retrieved = retrieval_pass(queryVec,incorp,lenRetrieve)
        
        retrievedout = retrieved.drop(columns=['reinfVec'])#.to_csv(queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        retrievedout.to_csv('/Users/eslt0101/Projects/EviDENce/ML/evaluation_data/single_pass_retrieval/'+queryTitle+'_'+str(lenRetrieve)+'.csv',index=False)
        
        pLR, rLR = calculate_PR_LR(retrieved)
        
        precisionLR.append(pLR)
        recallLR.append(rLR)
        queries.append(queryTitle)
        
        
    return precisionLR, recallLR, queries

In [156]:
precisionAtR, recallAtR, queryTitle = run_evaluation(groundTruthEx,182)

In [157]:
Data = {'Title': queryTitle, 'PR_at_R': precisionAtR }
results = pd.DataFrame(Data,columns=['Title','PR_at_R'])
results.sort_values(by='Title',inplace=True)

In [158]:
results.to_csv('/Users/eslt0101/Projects/EviDENce/ML/evaluation_data/single_pass_retrieval/evaluation_at_R.csv')